# Automatic Number Plate Recognition

In [1]:
import cv2
import numpy as np
import imutils
import easyocr

def process_video(file_path):
    def draw_pointer(frame, location, text):
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(frame, text=text, org=(location[0][0][0], location[1][0][1] + 60),
                    fontFace=font, fontScale=1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
        cv2.rectangle(frame, tuple(location[0][0]), tuple(location[2][0]), (0, 255, 0), 3)

    reader = easyocr.Reader(['en'])
    cap = cv2.VideoCapture(file_path)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        bfilter = cv2.bilateralFilter(gray, 11, 17, 17)  # Noise reduction
        edged = cv2.Canny(bfilter, 30, 200)  # Edge detection

        keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours = imutils.grab_contours(keypoints)
        contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

        location = None
        for contour in contours:
            approx = cv2.approxPolyDP(contour, 10, True)
            if len(approx) == 4:
                location = approx
                break

        if location is not None:
            mask = np.zeros(gray.shape, np.uint8)
            new_image = cv2.drawContours(mask, [location], 0, 255, -1)
            new_image = cv2.bitwise_and(frame, frame, mask=mask)

            (x, y) = np.where(mask == 255)
            (x1, y1) = (np.min(x), np.min(y))
            (x2, y2) = (np.max(x), np.max(y))
            cropped_image = gray[x1:x2 + 1, y1:y2 + 1]

            result = reader.readtext(cropped_image)
            if result:
                text = result[0][-2]
                draw_pointer(frame, location, text)

        cv2.imshow("Video Frame", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [2]:
process_video('video1.mp4')

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [3]:
import cv2
import numpy as np
import imutils
import easyocr

def process_video_alternative(file_path):
    def draw_highlight(frame, location, text):
        if location is not None:
            cv2.polylines(frame, [location], isClosed=True, color=(0, 255, 0), thickness=2)
            font = cv2.FONT_HERSHEY_SIMPLEX
            (x, y) = location[0][0]
            cv2.putText(frame, text, (x, y - 10), font, 0.8, (0, 255, 0), 2, cv2.LINE_AA)

    reader = easyocr.Reader(['en'])
    cap = cv2.VideoCapture(file_path)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        try:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            bfilter = cv2.bilateralFilter(gray, 11, 17, 17)  # Noise reduction
            edged = cv2.Canny(bfilter, 30, 200)  # Edge detection

            keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            contours = imutils.grab_contours(keypoints)
            contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

            location = None
            for contour in contours:
                approx = cv2.approxPolyDP(contour, 10, True)
                if len(approx) == 4:
                    location = approx
                    break

            if location is not None:
                mask = np.zeros(gray.shape, np.uint8)
                cv2.drawContours(mask, [location], -1, 255, -1)
                (x, y) = np.where(mask == 255)
                (x1, y1) = (np.min(x), np.min(y))
                (x2, y2) = (np.max(x), np.max(y))
                cropped_image = gray[x1:x2 + 1, y1:y2 + 1]

                result = reader.readtext(cropped_image)
                if result:
                    text = result[0][-2]
                    draw_highlight(frame, location, text)

        except Exception as e:
            print(f"Error processing frame: {e}")

        cv2.imshow("Number Plate Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [6]:
process_video_alternative('video3.mp4')

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
